## Content-based Recommender System
_Author: Rachel Koenig_
_____

Imports

In [1]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from bs4 import BeautifulSoup
from nltk.corpus import stopwords 

Read in Category csv.

In [2]:
categories = pd.read_csv('data/category_only.csv')
categories.head()

,Unnamed: 0,asin,name,Arts_Crafts_Sewing,Automotive,Baby_Products,Beauty_Personal_Care,Cell_Phones_Accessories,Clothing_Shoes_Jewelry,Electronics,...,Trunks,Umbrellas,Underwear,Wallets,Wear_to_Work,Wrist_Watches,Arm_Warmers,Baseball_Caps,Berets,overall_mean
0,0,0000031887,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.61
1,23,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17
2,29,1608299953,Learn French: Rosetta Stone French - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.31
3,42,1617160377,Learn Italian: Rosetta Stone Italian - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.62
4,58,B00001W0KA,Buzz Lightyear Boy's Deluxe Toy Story Costume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.62


In [3]:
categories.drop(columns='Unnamed: 0', inplace=True)  #drop unnamed column 
categories.head()  # check first 5 rows of the dataset 

,asin,name,Arts_Crafts_Sewing,Automotive,Baby_Products,Beauty_Personal_Care,Cell_Phones_Accessories,Clothing_Shoes_Jewelry,Electronics,Health_Household,...,Trunks,Umbrellas,Underwear,Wallets,Wear_to_Work,Wrist_Watches,Arm_Warmers,Baseball_Caps,Berets,overall_mean
0,0000031887,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.61
1,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17
2,1608299953,Learn French: Rosetta Stone French - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.31
3,1617160377,Learn Italian: Rosetta Stone Italian - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.62
4,B00001W0KA,Buzz Lightyear Boy's Deluxe Toy Story Costume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.62


In [4]:
# Check number of rows and columns 
categories.shape

(13732, 814)

In [5]:
# change all columns to lowercase 
categories.columns = [x.lower() for x in categories.columns]

In [110]:
# check data types 
categories.dtypes.head(25)

asin                        object
name                        object
arts_crafts_sewing         float64
automotive                 float64
baby_products              float64
beauty_personal_care       float64
cell_phones_accessories    float64
clothing_shoes_jewelry     float64
electronics                float64
health_household           float64
home_kitchen               float64
industrial_scientific      float64
office_products            float64
purchase_circles           float64
software                   float64
sports_outdoors            float64
tools_home_improvement     float64
toys_games                 float64
accessories_supplies       float64
antivirus_security         float64
arts_crafts                float64
baby_toddler_toys          float64
bath                       float64
beading_jewelry_making     float64
bedding                    float64
dtype: object

In [111]:
# Check for nulls 
categories.isnull().sum().sum()

0

In [9]:
# Check column/row output 
categories['name'][143]

"Leading Lady Women's Plus Size Sleep Leisure Cotton Bra"

Create new dataframe with only category columns.

In [10]:
cats = categories.iloc[:, 2:]
cats.head()

,arts_crafts_sewing,automotive,baby_products,beauty_personal_care,cell_phones_accessories,clothing_shoes_jewelry,electronics,health_household,home_kitchen,industrial_scientific,...,trunks,umbrellas,underwear,wallets,wear_to_work,wrist_watches,arm_warmers,baseball_caps,berets,overall_mean
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.61
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.31
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.62
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.62


Set index to product name.

In [11]:
cats = cats.set_index(categories['name'])

In [12]:
cats.head()

,arts_crafts_sewing,automotive,baby_products,beauty_personal_care,cell_phones_accessories,clothing_shoes_jewelry,electronics,health_household,home_kitchen,industrial_scientific,...,trunks,umbrellas,underwear,wallets,wear_to_work,wrist_watches,arm_warmers,baseball_caps,berets,overall_mean
name,,,,,,,,,,,,,,,,,,,,,
Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.61
SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / CASE / STORAGE / ORGANIZER WITH TRAVEL CASE AND LOCK,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17
Learn French: Rosetta Stone French - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.31
Learn Italian: Rosetta Stone Italian - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.62
Buzz Lightyear Boy's Deluxe Toy Story Costume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.62


In [13]:
cats.shape

(13732, 812)

In [14]:
cats['trunks'].value_counts()

0.0    13714
1.0       18
Name: trunks, dtype: int64

Convert to sparse matrix to make smaller file size 

In [18]:
categories_sparse = sparse.csr_matrix(cats.fillna(0))

Shape is still the same but each only cells with values are shown.

In [19]:
categories_sparse.shape

(13732, 812)

In [20]:
print(categories_sparse[:5])

  (0, 13)	1.0
  (0, 72)	1.0
  (0, 396)	1.0
  (0, 725)	1.0
  (0, 733)	1.0
  (0, 757)	1.0
  (0, 811)	4.61
  (1, 5)	1.0
  (1, 70)	1.0
  (1, 197)	1.0
  (1, 368)	1.0
  (1, 582)	1.0
  (1, 811)	4.17
  (2, 12)	1.0
  (2, 31)	1.0
  (2, 377)	1.0
  (2, 811)	4.31
  (3, 12)	1.0
  (3, 31)	1.0
  (3, 377)	1.0
  (3, 811)	4.62
  (4, 15)	1.0
  (4, 30)	1.0
  (4, 755)	1.0
  (4, 811)	4.62


Check similiarity of every item to every other item with pairwise distances. 

In [21]:
recommender = pairwise_distances(cats, metric='cosine')

In [22]:
#Checking to make sure it's a square. 
recommender.shape

(13732, 13732)

In [112]:
type(recommender)

numpy.ndarray

Convert to DataFrame with the index from categories as the index for both rows and columns.

In [24]:
recommender_df = pd.DataFrame(recommender, index=cats.index, columns=cats.index

In [25]:
# Check first 5 rows 
recommender_df.head()

name,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / CASE / STORAGE / ORGANIZER WITH TRAVEL CASE AND LOCK,Learn French: Rosetta Stone French - Level 1,Learn Italian: Rosetta Stone Italian - Level 1,Buzz Lightyear Boy's Deluxe Toy Story Costume,Woody Deluxe Child - Size: Child S(4-6),Lewis N. Clark Stash,"Lewis N. Clark Deluxe Neck Stash, Beige","Lewis N. Clark Add-A-Bag Travel Luggage Strap, Black, One Size",Buzz Lightyear Jet Pack,...,Sakkas Everyday Essentials Caftan Tank Dress/Cover Up,Sakkas Natalie Sequin Tie Dye Blouse,Sakkas Malvina Marbled Embroidery Cap Sleeves Blouse/Top,Lindy Bop 'Ophelia' Vintage 1950's Floral Spring Garden Party Picnic Dress,Free to Live Women's Fold Over High Waisted Flowy Floor Length Maxi Skirt,Rampage Womens Ultra Cute Embroidered Heather Jersey Night Shirt (Small-3X),14k Gold-Bonded Sterling Silver Tri-Color Hoop Earrings,Classic Designs Stretch Poplin Elastic Waist Cargo Capri,"TendzArt Azules Poly Span Floral Print Full Length Long Maxi Skirt - Made in USA (Medium, Purple+Mint+Yellow)","Kenneth Cole Reaction Easy To Remember, Black, One Size"
name,,,,,,,,,,,,,,,,,,,,,
Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0.000000,0.221747,0.180608,0.173118,0.173118,0.233270,0.188230,0.191571,0.210148,0.232939,...,0.166517,0.232997,0.187382,0.169129,0.113773,0.194185,0.206780,0.319598,0.130877,0.173562
SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / CASE / STORAGE / ORGANIZER WITH TRAVEL CASE AND LOCK,0.221747,0.000000,0.182269,0.174794,0.174794,0.182397,0.148275,0.150688,0.164491,0.187664,...,0.164311,0.229286,0.184627,0.166848,0.149660,0.191270,0.166848,0.314547,0.166071,0.132241
Learn French: Rosetta Stone French - Level 1,0.180608,0.182269,0.000000,0.000274,0.131173,0.194376,0.147051,0.150562,0.170081,0.194028,...,0.163110,0.246636,0.190082,0.166542,0.149071,0.198692,0.166542,0.349247,0.165493,0.131639
Learn Italian: Rosetta Stone Italian - Level 1,0.173118,0.174794,0.000274,0.000000,0.123232,0.187013,0.139255,0.142798,0.162495,0.186661,...,0.155461,0.239750,0.182679,0.158924,0.141293,0.191368,0.158924,0.343299,0.157866,0.123702
Buzz Lightyear Boy's Deluxe Toy Story Costume,0.173118,0.174794,0.131173,0.123232,0.000000,0.187013,0.139255,0.142798,0.162495,0.186661,...,0.155461,0.239750,0.182679,0.158924,0.141293,0.191368,0.158924,0.343299,0.157866,0.123702


### Ready to recommend!

A search query to find an exact item name

In [26]:
q = "Basic"
categories.loc[categories['name'].str.contains(q), 'name']

346      Russell Athletic Men's Basic Cotton Long Sleev...
482             Russell Athletic Men's Cotton Basic Muscle
484            Russell Athletic Men's Basic Cotton T-Shirt
490      Russell Athletic Men's Big & Tall Basic Crew N...
495      Russell Athletic Men's Big & Tall Basic Short ...
619      CTM Women's Elastic Clip-End 1 Inch Basic Susp...
1742       Capezio Women's Team Basic Short Sleeve Leotard
1744        Capezio Girls' Team Basic Short Sleeve Leotard
1746                Capezio Girls' Team Basic Tank Leotard
1747                Capezio Girls' Team Basic Tank Leotard
1748        Capezio Girls' Team Basics Long Sleeve Leotard
1876                            Timberland Basic Crew Sock
3238     Russell Athletic Men's Basic Cotton Pocket T-S...
3247     CTM Men's Leather 1 1/8 Inch Basic Dress Belt ...
3886               Luvable Friends 6 Pack Basic Cuff Socks
3952               Gold Toe Women's 3 Pk Basic Rib Trouser
3977             Southpole Juniors' Basic Solid Polo Shi

Search for a products index # 

In [124]:
categories[categories['name'] == "Learn French: Rosetta Stone French - Level 1"]

,asin,name,arts_crafts_sewing,automotive,baby_products,beauty_personal_care,cell_phones_accessories,clothing_shoes_jewelry,electronics,health_household,...,trunks,umbrellas,underwear,wallets,wear_to_work,wrist_watches,arm_warmers,baseball_caps,berets,overall_mean
2,1608299953,Learn French: Rosetta Stone French - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.31


In [125]:
foo = categories.loc[2].to_dict()

In [126]:
{k:v for k,v in foo.items() if v != 0}

{'asin': '1608299953',
 'name': 'Learn French: Rosetta Stone French - Level 1',
 'software': 1.0,
 'education_reference': 1.0,
 'languages': 1.0,
 'overall_mean': 4.31}

In [122]:
bar = categories.loc[2888].to_dict()

In [123]:
{k:v for k,v in bar.items() if v != 0}

{'asin': 'B001FYWBOI',
 'name': 'Elegance Long Stem Ring Holder, 9", Silver',
 'home_kitchen': 1.0,
 'home_décor': 1.0,
 'overall_mean': 5.0}

Top 10 recommendations for products using only categories as features.

In [113]:
recommender_df["Buzz Lightyear Boy's Deluxe Toy Story Costume"].sort_values()[1:11]


name_x
DC Super Heroes Child's Batgirl Costume, Large                                 0.167538
Classic Aviator Hat - Pilot Aviator Cap - Vintage WWII Hat Funny Party Hats    0.215495
Rubies Star Wars Classic Child's Hooded Jedi Robe, Large                       0.223021
Rieker Women's Celia 51                                                        0.231724
Super DC Heroes Wonder Woman Child's Costume                                   0.239798
Spiderman Muscle - Size: Child M(7-8)                                          0.242993
Forum Novelties Sparkle Princess Costume, Toddler Size                         0.245533
Disguise Brave Merida Wig                                                      0.253956
The Elder Wand, The Wand of Professor Dumbledore                               0.253998
New Balance KV20 Minimus Cross Training Shoe (Infant/Toddler)                  0.262818
Name: Buzz Lightyear Boy's Deluxe Toy Story Costume, dtype: float64

In [33]:
recommender_df["Learn French: Rosetta Stone French - Level 1"].sort_values()[1:11]

name
Learn Italian: Rosetta Stone Italian - Level 1                                               0.000274
Kaspersky Anti-Virus 2012 - 1 User [Old Version]                                             0.098442
Fitsok CF2 Cushion Low Cut Sock, 3-Pack                                                      0.107149
Bob Marley - Rasta Tam Womens T-Shirt in Asphalt                                             0.107149
Levi's Juniors Shortie Short                                                                 0.107149
Elegance Long Stem Ring Holder, 9", Silver                                                   0.107149
Tapp Collections trade; Mood Pendant Necklace - Cat                                          0.107149
Star Wars Anakin to Darth Vader Color Change Lightsaber Toy(Discontinued by manufacturer)    0.107149
Dakota Watch Company Mini Clip Microlight Watch                                              0.108093
Long Tulle Bustle Skirt with Satin Bow and Gather Lace Front                 

In [34]:
recommender_df["Russell Athletic Men's Basic Cotton T-Shirt"].sort_values()[1:11]

name
Hanes Ultimate Men's 3-Pack Classics Lightweight Slim-Fit V-Neck T-Shirt                          0.000506
Columbia Men's Low Drag Offshore Long Sleeve Shirt, UPF 40 Protection, Moisture Wicking Fabric    0.000927
Hanes Ultimate Men's 3-Pack Classics Lightweight Slim-Fit V-Neck T-Shirt                          0.001036
Noble Mount Mens 100% Cotton Casual Shirt - Regular Fit                                           0.019183
Charles River Apparel Men's Classic Rugby Shirt                                                   0.019183
Calvin Klein Men's Single Welt Pocket Polo                                                        0.019183
Russell Athletic Men's Big & Tall Dri Power Short-Sleeve Polo Shirt                               0.019183
Carhartt Men's Hines Solid Long-Sleeve Shirt Long-Sleeve Button-Front Twill                       0.019183
Carhartt Men's Chamois Button Front Original Fit Shirt                                            0.019183
PGA TOUR Men's Stacked Polo Shir

## Iterations to make model better 

Read in text csv with product names to count vectorize.

In [35]:
names = pd.read_csv('data/names_to_vectorize.csv')

In [114]:
# check first 5 rows 
names.head()

,asin,name,name_split
0,0000031887,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,mystiqueshapes girl ballet tutu neon lime green
1,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,shining image huge pink leather jewelry box ca...
2,1608299953,Learn French: Rosetta Stone French - Level 1,learn french rosetta stone french level 1
3,1617160377,Learn Italian: Rosetta Stone Italian - Level 1,learn italian rosetta stone italian level 1
4,B00001W0KA,Buzz Lightyear Boy's Deluxe Toy Story Costume,buzz lightyear boy deluxe toy story costume


In [37]:
# Check for nulls 
names.isnull().sum()

Unnamed: 0    0
asin          0
name          0
name_split    0
dtype: int64

In [38]:
# drop unnamed index column 
names.drop(columns='Unnamed: 0', inplace=True)

In [39]:
names.head()

,asin,name,name_split
0,0000031887,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,mystiqueshapes girl ballet tutu neon lime green
1,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,shining image huge pink leather jewelry box ca...
2,1608299953,Learn French: Rosetta Stone French - Level 1,learn french rosetta stone french level 1
3,1617160377,Learn Italian: Rosetta Stone Italian - Level 1,learn italian rosetta stone italian level 1
4,B00001W0KA,Buzz Lightyear Boy's Deluxe Toy Story Costume,buzz lightyear boy deluxe toy story costume


Read in colors csv.

In [40]:
colors = pd.read_csv('data/colors_split.csv')
colors.head()

,Unnamed: 0,asin,name,turquoise,nickel,diamond,cream,maize,sea green,dark green,...,copper,apricot,mustard,yellow,jade,bronze,cardinal,rose,golden brown,platinum
0,0,0000031887,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,23,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,29,1608299953,Learn French: Rosetta Stone French - Level 1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,42,1617160377,Learn Italian: Rosetta Stone Italian - Level 1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,58,B00001W0KA,Buzz Lightyear Boy's Deluxe Toy Story Costume,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
# drop unnamed index column 
colors.drop(columns=['Unnamed: 0', 'asin',], inplace=True)

In [42]:
# list of all colors from colors dataset to use as stopwords when CountVectorizing the names 
list_of_colors = ['acid green', 'aero', 'african violet', 'alabaster', 'alice blue', 'almond', 'amaranth', 'amazon', 'amber', 'amethyst',
 'antique brass', 'antique bronze', 'antique white', 'apple green', 'apricot', 'aqua', 'aquamarine', 'army green', 'ash grey',
 'auburn', 'avocado', 'awesome', 'azure', 'azure mist', 'baby blue', 'baby pink', 'banana yellow', 'barn red', 'begonia',
 'beige', 'bisque', 'bittersweet', 'black', 'black olive', 'blond', 'blue', 'blue bell', 'blue lagoon', 'blue sapphire',
 'blueberry', 'blush', 'bole', 'bondi blue', 'bone', 'boysenberry', 'brass', 'brick red', 'bright green', 'bright lavender',
 'bright pink', 'bronze', 'brown sugar', 'brown yellow', 'bubble gum', 'buff', 'burgundy', 'burnished brown', 'burnt orange', 'byzantine', 'cadet', 'cadet blue', 'camel',
 'camouflage green', 'canary', 'canary yellow', 'candy pink', 'capri', 'cardinal', 'carmine', 'carnelian', 'carolina blue', 'ceil',
 'celeste', 'celestial blue', 'cerise', 'cerulean', 'cerulean blue', 'champagne', 'charcoal', 'charm pink', 'cherry', 'chestnut', 'chinese red',
 'citrine', 'citron', 'cobalt blue', 'cocoa brown', 'coconut', 'coffee', 'columbia blue', 'cool grey', 'copper', 'copper red',
 'coral', 'coral pink', 'coral red', 'coral reef', 'cordovan', 'corn', 'cornflower blue', 'cornsilk', 'cotton candy', 'coyote brown',
 'cream', 'crimson', 'crimson red', 'cultured', 'cyan', 'cyber yellow', 'daffodil', 'dandelion', 'dark blue', 'dark brown',
 'dark coral', 'dark green', 'dark khaki', 'dark pink', 'dark purple', 'dark red', 'dark tan', 'dark taupe', 'dark yellow', 'deep fuchsia',
 'deep green', 'deep red', 'deer', 'denim', 'denim blue', 'desert', 'desert sand', 'desire', 'diamond', 'dirt', 'dodger blue', 'drab','ebony',
 'ecru', 'eggplant', 'eggshell', 'electric blue', 'electric green', 'electric lime', 'electric purple', 'electric yellow', 'emerald',
 'fandango', 'fawn', 'fern green', 'flame', 'flax', 'flirt', 'folly', 'french blue', 'french pink', 'french rose', 'frostbite', 'fuchsia', 'fuchsia purple',
 'ginger', 'glitter', 'go green', 'gold fusion', 'golden brown', 'golden yellow', 'grape', 'gray', 'green sheen', 'grizzly', 'gunmetal', 'harvest gold', 'heart gold',
 'hollywood cerise', 'hot pink', 'hunter green', 'iceberg', 'imperial', 'imperial blue', 'indigo', 'iris', 'irresistible', 'ivory',
 'jade', 'jasper', 'jet', 'jungle green', 'kelly green', 'kiwi', 'lapis lazuli', 'lava', 'lavender purple', 'lemon', 'lemon yellow', 'liberty',
 'light blue', 'light brown', 'light gray', 'light green', 'light pink', 'light taupe', 'light yellow', 'lilac', 'lime green',
 'linen', 'liver', 'lust', 'magenta', 'magnolia', 'mahogany', 'maize', 'malachite', 'mandarin', 'mango tango', 'mardi gras',
 'marigold', 'mauve', 'medium blue', 'medium purple', 'melon', 'midnight', 'midnight blue', 'milk','ming', 'mint', 'mint green', 'moccasin',
 'moss green', 'mulberry', 'mustard', 'mystic', 'navy', 'neon green', 'new car', 'nickel', 'ocean blue', 'ocean green', 'ochre', 'old gold',
 'old lavender', 'old rose', 'old silver', 'olive', 'onyx', 'orchid', 'oxford blue', 'pacific blue', 'pale green', 'pale pink',
 'paradise pink', 'pastel brown', 'pastel green', 'pastel pink', 'pastel red', 'peach', 'pear', 'pearl', 'peridot', 'periwinkle', 'persian red',
 'persimmon', 'peru', 'pine green', 'pink', 'pink flamingo','pink lace', 'pink pearl','pistachio', 'platinum', 'plum', 'popstar',
 'powder blue', 'pumpkin', 'purple heart', 'quartz', 'quick silver', 'raspberry', 'raspberry rose', 'red', 'redwood', 'regalia',
 'rhythm', 'rose', 'rose dust', 'rose gold', 'rose pink', 'rose quartz', 'rose red', 'rosewood', 'royal blue', 'royal purple',
 'ruby', 'ruby red', 'rust', 'safety orange', 'safety yellow', 'saffron', 'sage', 'salmon', 'sand', 'sand dune', 'sangria', 'sap green', 'sapphire', 'sapphire blue', 'scarlet',
 'sea blue', 'sea green', 'sepia', 'shadow', 'shadow blue', 'shamrock green', 'shocking pink', 'sienna', 'silver', 'silver pink', 'silver sand', 'sky blue', 'slate blue', 
 'slate gray', 'smitten', 'smoke', 'snow', 'solid pink',
 'spring green', 'steel blue', 'steel pink', 'straw', 'strawberry', 'sugar plum', 'sunny', 'sunset', 'sweet brown', 'tan', 'tangelo', 'tangerine', 'taupe', 'teal',
 'teal blue', 'teal green', 'terra cotta', 'thistle', 'timberwolf', 'titanium yellow', 'tomato', 'topaz', 'true blue', 'tulip', 'turquoise', 'turquoise blue', 
 'turquoise green', 'tuscan', 'tuscany', 'ua blue', 'ube', 'ultra pink', 'ultramarine', 'ultramarine blue', 'umber', 'vanilla', 'vegas gold', 'veronica', 
 'violet', 'vivid yellow', 'volt', 'wenge', 'wheat', 'white', 'wild strawberry', 'wine','yale blue','yellow','yellow rose']


In [43]:
type(list_of_colors)

list

In [44]:
colors['lime green'].value_counts()

0    13724
1        8
Name: lime green, dtype: int64

In [45]:
colors.head()

,name,turquoise,nickel,diamond,cream,maize,sea green,dark green,amber,jet,...,copper,apricot,mustard,yellow,jade,bronze,cardinal,rose,golden brown,platinum
0,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Learn French: Rosetta Stone French - Level 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Learn Italian: Rosetta Stone Italian - Level 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Buzz Lightyear Boy's Deluxe Toy Story Costume,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## CountVectorizer 

In [46]:
# Import CountVectorizer.
from sklearn.feature_extraction.text import CountVectorizer

In [131]:
# Instantiate the "CountVectorizer" object, which is scikit-learn's bag of words tool.
cvec = CountVectorizer(analyzer = "word",   # default, says features should be made of words 
                       tokenizer = None,    #default 
                       preprocessor = None, # default 
                       stop_words = list_of_colors,  # list of words I do not want counted 
                       max_features=10000,  # maximum number of features, n most frequent 
                       min_df=2,            # minimum number of documents a word must appear in   
                       ngram_range=(1, 3),  # a range of number of words to look at 
                       binary=True)         # returns a 1 or a 0 if a word is in the document 

In [132]:
names_features = cvec.fit_transform(names['name_split'])

/Users/rachelkoenig/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['acid', 'african', 'alice', 'antique', 'apple', 'army', 'ash', 'baby', 'banana', 'barn', 'bell', 'bondi', 'brick', 'bright', 'brown', 'bubble', 'burnished', 'burnt', 'camouflage', 'candy', 'car', 'carolina', 'celestial', 'charm', 'chinese', 'cobalt', 'cocoa', 'columbia', 'cool', 'cornflower', 'cotta', 'cotton', 'coyote', 'cyber', 'dark', 'deep', 'dodger', 'dune', 'dust', 'electric', 'fern', 'flamingo', 'french', 'fusion', 'go', 'gold', 'golden', 'gras', 'green', 'grey', 'gum', 'harvest', 'heart', 'hollywood', 'hot', 'hunter', 'jungle', 'kelly', 'khaki', 'lace', 'lagoon', 'lapis', 'lavender', 'lazuli', 'light', 'lime', 'mango', 'mardi', 'medium', 'mist', 'moss', 'neon', 'new', 'ocean', 'old', 'orange', 'oxford', 'pacific', 'pale', 'paradise', 'pastel', 'persian', 'pine', 'po

In [133]:
# check the shape 
names_features.shape

(13732, 10000)

In [130]:
type(names_features)

scipy.sparse.csr.csr_matrix

In [134]:
# Check the feature/column names 
vocab = cvec.get_feature_names()
print(vocab)

['00', '00 carat', '00 carat total', '00g', '01', '02', '03', '05', '0mm', '10', '10 13', '10 cttw', '10 cttw color', '10 inch', '10 pack', '10 pack cotton', '10 running', '10 running shoe', '10 watch', '10 year', '10 year battery', '100', '100 cashmere', '100 cashmere long', '100 cashmere neck', '100 cotton', '100 cotton flannel', '100 facet', '100 facet collection', '100 leather', '1000', '1000 running', '1000 running shoe', '100cm', '100pc', '100pc belly', '1050', '1050 hd', '1050 hd tuff', '10k', '10k gold', '10kt', '10kt gold', '10mm', '11', '11 12', '11 inch', '12', '12 14', '12 24', '12 inch', '12 mm', '12 month', '12 pack', '12 pair', '12 piece', '12 running', '12 running shoe', '120', '12mm', '13', '13 inch', '13 running', '13 running shoe', '14', '14 16', '14 16 18', '14 40', '14 40 inch', '14 running', '14 running shoe', '1440', '1440 sport', '1440 sport digital', '14g', '14k', '14k ball', '14k gold', '14k gold accent', '14k gold bonded', '14k gold filled', '14k gold hoop', 

Remove words that are just integers

In [135]:
keep_words = []     # instantiate an empty list 
for word in vocab:  # loop through the list words/strings
    try:
        [int(i) for i in word.split(" ")]  # try to turn each word in the string into an integer 
    except:                               # if it gets an error = not an integer
        keep_words.append(word)           # add it tot he keep words list

In [136]:
keep_words

['00 carat',
 '00 carat total',
 '00g',
 '0mm',
 '10 cttw',
 '10 cttw color',
 '10 inch',
 '10 pack',
 '10 pack cotton',
 '10 running',
 '10 running shoe',
 '10 watch',
 '10 year',
 '10 year battery',
 '100 cashmere',
 '100 cashmere long',
 '100 cashmere neck',
 '100 cotton',
 '100 cotton flannel',
 '100 facet',
 '100 facet collection',
 '100 leather',
 '1000 running',
 '1000 running shoe',
 '100cm',
 '100pc',
 '100pc belly',
 '1050 hd',
 '1050 hd tuff',
 '10k',
 '10k gold',
 '10kt',
 '10kt gold',
 '10mm',
 '11 inch',
 '12 inch',
 '12 mm',
 '12 month',
 '12 pack',
 '12 pair',
 '12 piece',
 '12 running',
 '12 running shoe',
 '12mm',
 '13 inch',
 '13 running',
 '13 running shoe',
 '14 40 inch',
 '14 running',
 '14 running shoe',
 '1440 sport',
 '1440 sport digital',
 '14g',
 '14k',
 '14k ball',
 '14k gold',
 '14k gold accent',
 '14k gold bonded',
 '14k gold filled',
 '14k gold hoop',
 '14k gold plated',
 '14k gold round',
 '14mm',
 '16 extender',
 '16 inch',
 '16g',
 '17 inch',
 '18 hour

Turn newly created feaures matrix into a DataFrame.

In [137]:
feature_names_df = pd.DataFrame(names_features.toarray(),
                         columns= cvec.get_feature_names())

In [138]:
feature_names_df.head()

,00,00 carat,00 carat total,00g,01,02,03,05,0mm,10,...,zirconia halo,zirconia pendant,zirconia pendant necklace,zirconia ring,zirconia round,zirconia stud,zirconia stud earring,zombie,zoned,zoned support
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
# Check value counts of a specific feature 
feature_names_df['zombie'].value_counts()

0    13726
1        6
Name: zombie, dtype: int64

In [140]:
# Find the column names that intersect with the words in the keep list 
feature_names_df.columns.intersection(keep_words)

Index(['00 carat', '00 carat total', '00g', '0mm', '10 cttw', '10 cttw color',
       '10 inch', '10 pack', '10 pack cotton', '10 running',
       ...
       'zirconia halo', 'zirconia pendant', 'zirconia pendant necklace',
       'zirconia ring', 'zirconia round', 'zirconia stud',
       'zirconia stud earring', 'zombie', 'zoned', 'zoned support'],
      dtype='object', length=9885)

In [141]:
# Create a DataFrame with the just the non-integer words 
feature_names_df = feature_names_df[feature_names_df.columns.intersection(keep_words)]

In [143]:
# Check features for one row
box = feature_names_df.loc[1].to_dict()

{k:v for k,v in box.items() if v != 0}

{'box': 1,
 'box case': 1,
 'case': 1,
 'case lock': 1,
 'jewelry': 1,
 'jewelry box': 1,
 'jewelry box case': 1,
 'leather': 1,
 'leather jewelry': 1,
 'leather jewelry box': 1,
 'lock': 1,
 'organizer': 1,
 'organizer travel': 1,
 'organizer travel case': 1,
 'storage': 1,
 'storage organizer': 1,
 'storage organizer travel': 1,
 'travel': 1,
 'travel case': 1,
 'travel case lock': 1}

In [144]:
# Check number of rows and columns 
feature_names_df.shape

(13732, 9885)

In [145]:
feature_names_df.head()

,00 carat,00 carat total,00g,0mm,10 cttw,10 cttw color,10 inch,10 pack,10 pack cotton,10 running,...,zirconia halo,zirconia pendant,zirconia pendant necklace,zirconia ring,zirconia round,zirconia stud,zirconia stud earring,zombie,zoned,zoned support
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Add name features 

In [146]:
cats_feats = pd.merge(categories, feature_names_df, right_index=True, left_index=True)

In [147]:
cats_feats.head()

,asin,name_x,arts_crafts_sewing,automotive,baby_products,beauty_personal_care,cell_phones_accessories,clothing_shoes_jewelry,electronics,health_household,...,zirconia halo,zirconia pendant,zirconia pendant necklace,zirconia ring,zirconia round,zirconia stud,zirconia stud earring,zombie,zoned,zoned support
0,0000031887,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1608299953,Learn French: Rosetta Stone French - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1617160377,Learn Italian: Rosetta Stone Italian - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,B00001W0KA,Buzz Lightyear Boy's Deluxe Toy Story Costume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [148]:
cats_feats.shape

(13732, 10699)

In [149]:
cats_feats = cats_feats.set_index('name_x').drop(columns='asin')

In [150]:
cats_feats.shape

(13732, 10697)

In [151]:
cats_feats_sparse = sparse.csr_matrix(cats_feats.fillna(0))

In [152]:
recommender2 = pairwise_distances(cats_feats, metric='cosine')

In [153]:
recommender_df = pd.DataFrame(recommender2, index=cats_feats.index, columns=cats_feats.index)
recommender_df.head()

name_x,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / CASE / STORAGE / ORGANIZER WITH TRAVEL CASE AND LOCK,Learn French: Rosetta Stone French - Level 1,Learn Italian: Rosetta Stone Italian - Level 1,Buzz Lightyear Boy's Deluxe Toy Story Costume,Woody Deluxe Child - Size: Child S(4-6),Lewis N. Clark Stash,"Lewis N. Clark Deluxe Neck Stash, Beige","Lewis N. Clark Add-A-Bag Travel Luggage Strap, Black, One Size",Buzz Lightyear Jet Pack,...,Sakkas Everyday Essentials Caftan Tank Dress/Cover Up,Sakkas Natalie Sequin Tie Dye Blouse,Sakkas Malvina Marbled Embroidery Cap Sleeves Blouse/Top,Lindy Bop 'Ophelia' Vintage 1950's Floral Spring Garden Party Picnic Dress,Free to Live Women's Fold Over High Waisted Flowy Floor Length Maxi Skirt,Rampage Womens Ultra Cute Embroidered Heather Jersey Night Shirt (Small-3X),14k Gold-Bonded Sterling Silver Tri-Color Hoop Earrings,Classic Designs Stretch Poplin Elastic Waist Cargo Capri,"TendzArt Azules Poly Span Floral Print Full Length Long Maxi Skirt - Made in USA (Medium, Purple+Mint+Yellow)","Kenneth Cole Reaction Easy To Remember, Black, One Size"
name_x,,,,,,,,,,,,,,,,,,,,,
Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0.000000,0.487963,0.331621,0.318178,0.306255,0.362259,0.316190,0.343528,0.434703,0.322028,...,0.347123,0.415019,0.394512,0.369880,0.339834,0.442427,0.428619,0.566207,0.387438,0.370798
SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / CASE / STORAGE / ORGANIZER WITH TRAVEL CASE AND LOCK,0.487963,0.000000,0.464522,0.453753,0.444200,0.454061,0.424028,0.446342,0.492804,0.423608,...,0.474492,0.528113,0.512275,0.492760,0.491489,0.550769,0.518212,0.649172,0.528157,0.469630
Learn French: Rosetta Stone French - Level 1,0.331621,0.464522,0.000000,0.036302,0.274496,0.333064,0.284886,0.313475,0.408824,0.290991,...,0.347541,0.428127,0.399365,0.370894,0.369113,0.448156,0.402461,0.587063,0.414605,0.341993
Learn Italian: Rosetta Stone Italian - Level 1,0.318178,0.453753,0.036302,0.000000,0.259905,0.319651,0.270503,0.299668,0.396935,0.276731,...,0.334419,0.416625,0.387285,0.358242,0.356425,0.437058,0.390443,0.578758,0.402832,0.328759
Buzz Lightyear Boy's Deluxe Toy Story Costume,0.306255,0.444200,0.274496,0.259905,0.000000,0.264943,0.257747,0.253523,0.386389,0.264083,...,0.322780,0.406424,0.376570,0.347019,0.345170,0.427214,0.379784,0.571392,0.392389,0.317021


In [154]:
recommender_df.shape

(13732, 13732)

In [159]:
q = "Van Authentic"
categories.loc[categories['name'].str.contains(q), 'name']

2887    Van Authentic
Name: name, dtype: object

In [156]:
recommender_df["Buzz Lightyear Boy's Deluxe Toy Story Costume"].sort_values()[1:11]


name_x
Storybook Cinderella Prestige                                 0.089651
California Costumes Toys Hercules                             0.090486
California Costumes Toys Vampire Girl                         0.106665
The Elder Wand, The Wand of Professor Dumbledore              0.124530
Child's Red and Black Spanish Princess Costume, Small         0.127036
Rubie's Deluxe George Washington Children's Costume, Large    0.135076
Disney Frozen Enchanting Dress - Anna                         0.135247
Thor Child Movie Hammer                                       0.135763
Spiderman Muscle - Size: Child M(7-8)                         0.146564
DC Super Heroes Child's Batgirl Costume, Large                0.153396
Name: Buzz Lightyear Boy's Deluxe Toy Story Costume, dtype: float64

In [157]:
recommender_df['SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / CASE / STORAGE / ORGANIZER WITH TRAVEL CASE AND LOCK'].sort_values()[1:11]

name_x
Kendal Leather Jewelry Box CASE Storage Organizer with Travel CASE and Lock (Purple)    0.023412
Kendal Brown LEATHER JEWELRY BOX CASE STORAGE ORGANIZER WITH TRAVEL CASE AND LOCK       0.032362
Black Leather Jewelry Box Travel Case and Lock                                          0.100190
Elegant Rosewood Finish Jewelry Box Case Holder Tray                                    0.233831
Enchantmints Unicorn Music Jewelry Box                                                  0.239869
Orb Factory Sticky Mosaics: Jewelry Box                                                 0.239991
Fairy Tale Jewelry Box                                                                  0.240468
1 X Bronze Tone Embossed Flower Old Stye Wooden Jewelry Box Case                        0.243805
Lenox Childhood Memories Ballerina Jewelry Box                                          0.252077
Enchantmints Horse Ranch Music Jewelry Box                                              0.252211
Name: SHINING IMAGE HUG

In [158]:
recommender_df["Vans Men's Authentic(tm) Core Classics"].sort_values()[1:11]



name_x
Van Authentic                     0.063713
Vans Slip-on(tm) Core Classics    0.127692
Superga 2750 Cotu Classic 2       0.139120
UGG W Maylin Chestnut             0.147385
Zumba Women's Z1-W                0.147682
DC Women's Aubrey Sneaker         0.157860
Chaco Women's Updraft Bulloo      0.160657
Rieker Women's Celia 51           0.161716
Chaco Women's Z/1 Yampa Sandal    0.161831
Birki's Women's Classic           0.162952
Name: Vans Men's Authentic(tm) Core Classics, dtype: float64

In [107]:
categories[categories['name'] == "Dansko Women's Professional Pull-Up Clog"]

,asin,name,arts_crafts_sewing,automotive,baby_products,beauty_personal_care,cell_phones_accessories,clothing_shoes_jewelry,electronics,health_household,...,trunks,umbrellas,underwear,wallets,wear_to_work,wrist_watches,arm_warmers,baseball_caps,berets,overall_mean
2799,B001EJMXRE,Dansko Women's Professional Pull-Up Clog,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17


In [160]:
foo = feature_names_df.loc[2887].to_dict()

{k:v for k,v in foo.items() if v != 0}

{'authentic': 1, 'van': 1}

In [161]:
foo = categories.loc[2887].to_dict()

{k:v for k,v in foo.items() if v != 0}

{'asin': 'B001FYJ5J2',
 'name': 'Van Authentic',
 'clothing_shoes_jewelry': 1.0,
 'skateboarding': 1.0,
 'women': 1.0,
 'shoes': 1.0,
 'athletic': 1.0,
 'overall_mean': 4.55}

### Add color features

In [78]:
colors.head()

,name,turquoise,nickel,diamond,cream,maize,sea green,dark green,amber,jet,...,copper,apricot,mustard,yellow,jade,bronze,cardinal,rose,golden brown,platinum
0,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Learn French: Rosetta Stone French - Level 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Learn Italian: Rosetta Stone Italian - Level 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Buzz Lightyear Boy's Deluxe Toy Story Costume,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
categories.head()

,asin,name,arts_crafts_sewing,automotive,baby_products,beauty_personal_care,cell_phones_accessories,clothing_shoes_jewelry,electronics,health_household,...,trunks,umbrellas,underwear,wallets,wear_to_work,wrist_watches,arm_warmers,baseball_caps,berets,overall_mean
0,0000031887,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.61
1,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17
2,1608299953,Learn French: Rosetta Stone French - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.31
3,1617160377,Learn Italian: Rosetta Stone Italian - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.62
4,B00001W0KA,Buzz Lightyear Boy's Deluxe Toy Story Costume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.62


In [80]:
cats_colors = pd.merge(left=categories, right=colors, on='name') 

In [81]:
cats_colors_names = pd.merge(cats_colors, feature_names_df, right_index=True, left_index=True)

In [82]:
print(cats_colors_names.shape)
cats_colors_names.head()

(13732, 24161)


,asin,name_x,arts_crafts_sewing,automotive,baby_products,beauty_personal_care,cell_phones_accessories,clothing_shoes_jewelry,electronics,health_household,...,zodiac,zombie,zone,zoned,zoned support,zoned support full,zoned support underwire,zumba,zutano,zuzu
0,0000031887,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1608299953,Learn French: Rosetta Stone French - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1617160377,Learn Italian: Rosetta Stone Italian - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,B00001W0KA,Buzz Lightyear Boy's Deluxe Toy Story Costume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
df = cats_colors_names.set_index('name_x').drop(columns='asin')

In [84]:
df.head()

,arts_crafts_sewing,automotive,baby_products,beauty_personal_care,cell_phones_accessories,clothing_shoes_jewelry,electronics,health_household,home_kitchen,industrial_scientific,...,zodiac,zombie,zone,zoned,zoned support,zoned support full,zoned support underwire,zumba,zutano,zuzu
name_x,,,,,,,,,,,,,,,,,,,,,
Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / CASE / STORAGE / ORGANIZER WITH TRAVEL CASE AND LOCK,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
Learn French: Rosetta Stone French - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
Learn Italian: Rosetta Stone Italian - Level 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
Buzz Lightyear Boy's Deluxe Toy Story Costume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
df_sparse = sparse.csr_matrix(df.fillna(0))

In [86]:
df_sparse.shape

(13732, 24159)

In [87]:
recommender3 = pairwise_distances(df_sparse, metric='cosine')

In [88]:
recommender3.shape

(13732, 13732)

In [89]:
recommender_df = pd.DataFrame(recommender3, index=df.index, columns=df.index)
recommender_df.head()

name_x,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / CASE / STORAGE / ORGANIZER WITH TRAVEL CASE AND LOCK,Learn French: Rosetta Stone French - Level 1,Learn Italian: Rosetta Stone Italian - Level 1,Buzz Lightyear Boy's Deluxe Toy Story Costume,Woody Deluxe Child - Size: Child S(4-6),Lewis N. Clark Stash,"Lewis N. Clark Deluxe Neck Stash, Beige","Lewis N. Clark Add-A-Bag Travel Luggage Strap, Black, One Size",Buzz Lightyear Jet Pack,...,Steve Madden Men's Essexx Lace-Up Boot,U.S. Polo Assn. Sport Men's US9117 Black Silicone Analog Digital Watch,U.S. Polo Assn. Sport Men's US9175 Black Ana-Digi Watch with Blue Silicone Band,U.S. Polo Assn. Sport Men's US9173 Sport Watch,DV by Dolce Vita Women's Sargeant Boot,Amy Byer Little Girls' Asymmetrical Sequin Dress,Van Heusen Men's Dress Shirts Regular Fit Lux Sateen Solid Spread Collar,Van Heusen Men's Regular Fit Poplin Solid Spread Collar Dress Shirt,"YSense 5 Pairs Womens Funny Casual Cotton Crew Socks (14, one Size)",Aerosoles Women's Sawcremento
name_x,,,,,,,,,,,,,,,,,,,,,
Mystiqueshapes Girls Ballet Tutu Neon Lime Green,0.000000,0.529666,0.384660,0.359277,0.369259,0.410482,0.335871,0.372559,0.496179,0.395683,...,0.412663,0.358667,0.497645,0.382913,0.462502,0.406242,0.406242,0.384400,0.479676,0.403522
SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / CASE / STORAGE / ORGANIZER WITH TRAVEL CASE AND LOCK,0.529666,0.000000,0.519930,0.500127,0.507915,0.508563,0.455260,0.484689,0.559803,0.499687,...,0.515426,0.474962,0.581971,0.539937,0.557219,0.534654,0.534654,0.517536,0.571577,0.511396
Learn French: Rosetta Stone French - Level 1,0.384660,0.519930,0.000000,0.050422,0.356203,0.398279,0.322123,0.359570,0.485749,0.383173,...,0.400505,0.345391,0.487246,0.398097,0.451375,0.420155,0.420155,0.398825,0.468905,0.391174
Learn Italian: Rosetta Stone Italian - Level 1,0.359277,0.500127,0.050422,0.000000,0.329646,0.373457,0.294161,0.333152,0.464536,0.357729,...,0.375776,0.318388,0.466095,0.373268,0.428744,0.396236,0.396236,0.374026,0.446997,0.366060
Buzz Lightyear Boy's Deluxe Toy Story Costume,0.369259,0.507915,0.356203,0.329646,0.000000,0.345075,0.305158,0.312313,0.472879,0.263000,...,0.385501,0.329007,0.474413,0.383033,0.437645,0.405643,0.405643,0.383779,0.455613,0.375937


## TF-IDF

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(colors_and_names['name_split'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


In [ ]:
print(tfidf.get_feature_names())

In [ ]:
feature_names_tfidf = pd.DataFrame(tfidf_matrix.toarray(),
                         columns= tfidf.get_feature_names())

In [ ]:
feature_names_tfidf.head()

In [ ]:
df2 = pd.merge(categories, feature_names_df)

In [ ]:
df2.head()